# Case study - zbiór `adults`

In [1]:
# !pip install pyspark
import pyspark

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as f

spark = SparkSession.builder.appName('DataFrame').master('local[*]').getOrCreate()

23/08/12 18:25:22 WARN Utils: Your hostname, md-ASUS resolves to a loopback address: 127.0.1.1; using 192.168.30.211 instead (on interface eth0)
23/08/12 18:25:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/12 18:25:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/12 18:25:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
col_names = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation", "relationship",
             "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "earnings"]

df = spark.read.csv("adult.data", header=False, inferSchema=True, ignoreLeadingWhiteSpace=True)

df = df.select(*[f.col(old).alias(new) for old, new in zip(df.columns, col_names)]).drop("fnlwgt").dropna("any")

df.show(3, vertical=True)

-RECORD 0----------------------------
 age            | 39                 
 workclass      | State-gov          
 education      | Bachelors          
 education-num  | 13                 
 marital-status | Never-married      
 occupation     | Adm-clerical       
 relationship   | Not-in-family      
 race           | White              
 sex            | Male               
 capital-gain   | 2174               
 capital-loss   | 0                  
 hours-per-week | 40                 
 native-country | United-States      
 earnings       | <=50K              
-RECORD 1----------------------------
 age            | 50                 
 workclass      | Self-emp-not-inc   
 education      | Bachelors          
 education-num  | 13                 
 marital-status | Married-civ-spouse 
 occupation     | Exec-managerial    
 relationship   | Husband            
 race           | White              
 sex            | Male               
 capital-gain   | 0                  
 capital-los

In [32]:
df.toPandas()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,earnings
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


##  ZADANIA

> 1. Oblicz średnią `capital-gain` oraz `capital-loss` dla każdej narodowości - wynik uporządkuj malejąco po średnim zysku

In [37]:
df.groupby("native-country") \
    .agg(f.round(f.avg("capital-gain")).alias("avg_gain"), f.round(f.avg("capital-loss")).alias("avg_loss")) \
    .sort("avg_gain", ascending=False) \
    .show(20)

+------------------+--------+--------+
|    native-country|avg_gain|avg_loss|
+------------------+--------+--------+
|             India|  3605.0|   131.0|
|             Japan|  2546.0|    58.0|
|            Taiwan|  2147.0|   155.0|
|              Iran|  2045.0|   127.0|
|             South|  1870.0|   169.0|
|                 ?|  1807.0|   118.0|
|            Canada|  1504.0|   130.0|
|Dominican-Republic|  1476.0|    25.0|
|            Greece|  1284.0|   295.0|
|     United-States|  1089.0|    89.0|
|          Cambodia|  1028.0|   183.0|
|       Philippines|  1018.0|    99.0|
|           Germany|   887.0|    78.0|
|           England|   865.0|    83.0|
|           Hungary|   717.0|   128.0|
|            Poland|   684.0|    34.0|
|           Vietnam|   661.0|    87.0|
|             Italy|   612.0|    27.0|
|       El-Salvador|   574.0|    31.0|
|            France|   547.0|     0.0|
+------------------+--------+--------+
only showing top 20 rows



> 2. Oblicz jaki procent kobiet zarabia ponad 50K w podziale na wykształcenie - wynik zaokrąglij do 2 miejsc po przecinku

In [21]:
df.filter(df.sex == "Female") \
    .withColumn("is_rich", f.when(f.col("earnings") == ">50K", 1).otherwise(0)) \
    .groupby("education") \
    .agg(f.round(f.avg("is_rich"), 2).alias("pct_rich")) \
    .show()

+------------+--------+
|   education|pct_rich|
+------------+--------+
|     Masters|    0.33|
|        10th|    0.01|
|     5th-6th|    0.02|
|  Assoc-acdm|    0.13|
|   Assoc-voc|    0.13|
|     7th-8th|    0.01|
|         9th|    0.03|
|     HS-grad|    0.07|
|   Bachelors|    0.21|
|        11th|    0.02|
|     1st-4th|     0.0|
|   Preschool|     0.0|
|        12th|    0.03|
|   Doctorate|    0.58|
|Some-college|    0.07|
| Prof-school|    0.46|
+------------+--------+



> 3. Oblicz ile godzin średnio w ciągu tygodnia pracują osoby `Never-married` w stosunku do reszty społeczeństwa

In [35]:
df.withColumn("marriage_group", f.when(f.col("marital-status") == "Never-married", "Never-married").otherwise("Others")) \
    .groupby("marriage_group") \
    .agg(f.round(f.avg("hours-per-week")).alias("avg_hours")) \
    .show()

+--------------+---------+
|marriage_group|avg_hours|
+--------------+---------+
| Never-married|     37.0|
|        Others|     42.0|
+--------------+---------+



> 4. W jakim wieku najczęściej zarabia sie powyżej 50K?

In [42]:
df.withColumn("is_rich", f.when(f.col("earnings") == ">50K", 1).otherwise(0)) \
    .groupby("age") \
    .agg(f.round(f.avg("is_rich"), 2).alias("pct_rich")) \
    .sort("pct_rich", ascending=False) \
    .show(5)

+---+--------+
|age|pct_rich|
+---+--------+
| 50|    0.43|
| 54|    0.42|
| 47|    0.41|
| 53|    0.41|
| 51|    0.41|
+---+--------+
only showing top 5 rows



> 5. W podziale ze względu na zmienną `race` oblicz stosunek liczby kobiet do mężczyzn

In [57]:
df.groupby("race", "sex").count().sort("race").show()

df.withColumns({"is_male": f.when(f.col("sex") == "Male", 1).otherwise(0),
               "is_female": f.when(f.col("sex") == "Female", 1).otherwise(0)}) \
    .groupby("race") \
    .agg(f.round(f.sum("is_female") / f.sum("is_male"), 2).alias("female / male")) \
    .show()

+------------------+------+-----+
|              race|   sex|count|
+------------------+------+-----+
|Amer-Indian-Eskimo|  Male|  192|
|Amer-Indian-Eskimo|Female|  119|
|Asian-Pac-Islander|  Male|  693|
|Asian-Pac-Islander|Female|  346|
|             Black|  Male| 1569|
|             Black|Female| 1555|
|             Other|  Male|  162|
|             Other|Female|  109|
|             White|Female| 8642|
|             White|  Male|19174|
+------------------+------+-----+

+------------------+-------------+
|              race|female / male|
+------------------+-------------+
|             Other|         0.67|
|Amer-Indian-Eskimo|         0.62|
|             White|         0.45|
|Asian-Pac-Islander|          0.5|
|             Black|         0.99|
+------------------+-------------+



> 6. Oblicz różnicę pomiędzy średnim czasem pracy osób uzyskujących `capital-gain` a resztą populacji

In [88]:
df2 = df.withColumn("has_capital_gain", f.when(f.col("capital-gain") > 0, "True").otherwise("False")) \
            .groupby("has_capital_gain") \
            .agg(f.round(f.avg("hours-per-week"), 2).alias("avg_hours")) \
            .sort("avg_hours", ascending=False).toPandas()

print(df2)

print(f'\nróżnica: {round(df2.avg_hours[0] - df2.avg_hours[1], 2)}')

  has_capital_gain  avg_hours
0             True      43.51
1            False      40.16

różnica: 3.35


In [ ]:
# inne rozwiązanie:

df2 = df.withColumn("has_capital_gain", f.when(f.col("capital-gain") > 0, "True").otherwise("False")) \
            .groupby("has_capital_gain") \
            .agg(f.round(f.avg("hours-per-week"), 2).alias("avg_hours")) \
            .sort("avg_hours", ascending=False).collect()

print(f'\nróżnica: {round(df2[0].avg_hours - df2[1].avg_hours, 2)}')